In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert')

tokens = ["dyspnea", "fagner", "jessica"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
word_embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# word_embedding_model.save('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output')

In [ ]:
# len(word_embedding_model.tokenizer)

In [ ]:
import csv, logging

from sentence_transformers import InputExample


# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

sts_dataset_path = '/content/drive/MyDrive/Colab Notebooks/datasets/snli/'


train_samples = []
dev_samples = []
test_samples = []

with open(sts_dataset_path + 'snli_1.0_train.txt', "r") as f:
#     labels = f.readlines()
#     print(f.readlines())
    reader = csv.DictReader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
#         print(row)
#         score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1

        gold_label = row['gold_label']  # Normalize score to range 0 ... 1
        float_gold_label = 0
        
        if gold_label == 'contradiction':
            float_gold_label = 0 / 3.0
        if gold_label == 'entailment':
            float_gold_label = 1 / 3.0
        if gold_label == 'neutral':
            float_gold_label = 2 / 3.0
        
        
#         print(gold_label)
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=float_gold_label)

#         if row['split'] == 'dev':
        train_samples.append(inp_example)
#         elif row['split'] == 'test':
#             test_samples.append(inp_example)
#         else:
#             train_samples.append(inp_example)

    
print(train_samples[0])
# with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
# reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)

<InputExample> label: 0.6666666666666666, texts: A person on a horse jumps over a broken down airplane.; A person is training his horse for a competition.


In [ ]:
import math 

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from sentence_transformers import losses


num_epochs = 3
train_batch_size = 32

model_save_path = '/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output'

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=word_embedding_model)


# Development set: Measure correlation between cosine score and gold labels
logging.info("Read SNLIbenchmark dev dataset")
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
word_embedding_model.fit(train_objectives=[(train_dataloader, train_loss)],
#           evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

# model = SentenceTransformer(model_save_path)
# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
# test_evaluator(model, output_path=model_save_path)


In [ ]:
! nvidia-smi

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 1.9MB 29.6MB/s 
     |████████████████████████████████| 1.2MB 52.9MB/s 
     |████████████████████████████████| 3.2MB 50.3MB/s 
     |████████████████████████████████| 890kB 52.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=e9588377828963d6b33fccb50cf7a5f26d55f84a9477099e30243def3505d9bb
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b64a9f94fc01a4b6b60f62644bf49a9603789e0da71e35eb565f960a7d3c5977
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer, models

model_save_path = '/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output'

# model = SentenceTransformer(model_save_path)

word_embedding_model.save('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output')

In [ ]:
import pdb
import torch

md = torch.load('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output/0_Transformer/pytorch_model.bin',map_location='cuda:0')

for k in md:
# modelos finetunados tão vindo sem o prefixo bert.
    if (k == 'bert.embeddings.word_embeddings.weight' or k == 'embeddings.word_embeddings.weight'):
        embeds = md[k]

vectors = []

for l in range(len(embeds)):
    vector = embeds[l]
    tsv_row = ''
    for m in range(len(vector)):
        tsv_row += str(vector[m].tolist()) + '\t'

    vectors.append(tsv_row)

print(len(vectors))




28999


In [ ]:
added_vocab = word_embedding_model.tokenizer.get_added_vocab()

with open('/content/drive/MyDrive/Colab Notebooks/tsv_files/clinical_bert/output/word_embeddings.tsv', "w") as f, open('/content/drive/MyDrive/Colab Notebooks/tsv_files/clinical_bert/output/labels.tsv', "a") as labels_file:
    i = 0
    for e in vectors:
#         print(i)
        i = i+1
        try:
            label = list(added_vocab.keys())[list(added_vocab.values()).index(i)]  # Prints george
            print('token novo')

            print(label)
            print (e, file=f)
            
            labels_file.write(label+'\n')

        except:
            # labels_file.write(label+'\n')

            print (e, file=f)


token novo
dyspnea
token novo
fagner
token novo
jessica


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

print(str(n_gpu)+ ' '+ str(device) + ' Device available: '+ torch.cuda.get_device_name(0))

1 cuda Device available: Tesla T4


In [ ]:
import torch

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output')
# md = torch.load('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output/0_Transformer/pytorch_model.bin',map_location='cuda:0')


In [ ]:
model

SentenceTransformer(
  (0): Transformer(
    (auto_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28999, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
       